# Set-up

In [150]:
import math 
from typing import List, Tuple, Callable
import numpy as np
import scipy.stats as sp
import pandas as pd
import statsmodels.api as sm
import statsmodels.stats.api as sms
import statsmodels.formula.api as smf
from sklearn.linear_model import LinearRegression

# 1. Linear regression

## Linear algebra

### Vectors

In [108]:
Vector = List[float]

height_weight_age = [70,170,40]
grades = [95,80,75,62]

In [111]:
def add(v: Vector, w: Vector) -> Vector:
    '''Adds corresponding elements'''
    assert len(v) == len(w), 'vectors must be the same length'
    return [v_i + w_i for v_i, w_i in zip(v, w)]
assert add([1,2,3], [4,5,6]) == [5,7,9]

In [112]:
def subtract(v: Vector, w: Vector) -> Vector:
    '''Subtracts corresponsing elements'''
    assert len(v) == len(w), 'vectors must be the same length'
    return [v_i - w_i for v_i, w_i in zip(v, w)]
assert subtract([5,7,9], [4,5,6]) == [1,2,3]

In [113]:
def vector_sum(vectors: List[Vector]) -> Vector:
    '''Sums all corresponding elements'''
    # check that vectors is not empty 
    assert vectors, 'no vectors provided'
    
    # check the vectors are all the same size
    num_elements = len(vectors[0])
    assert all(len(v) == num_elements for v in vectors), 'different sizes'
    
    # the i-th element of the result is the sum of every vector[i]
    return [sum(vector[i] for vector in vectors)
            for i in range(num_elements)]
assert vector_sum([[1,2],[3,4],[5,6],[7,8]]) == [16,20]

In [115]:
sum([np.array([1,2]),np.array([3,4]),np.array([5,6]),np.array([7,8])])

array([16, 20])

In [116]:
def scalar_multiply(c: float, v: Vector) -> Vector:
    '''Multiplies every element by c'''
    return [c * v_i for v_i in v]
assert scalar_multiply(2, [1,2,3]) == [2,4,6]

In [117]:
np.array([1,2,3]) * 2

array([2, 4, 6])

In [119]:
def vector_mean(vectors: List[Vector]) -> Vector:
    '''Computes the element-wise average'''
    n = len(vectors)
    return scalar_multiply(1/n, vector_sum(vectors))
assert vector_mean([[1,2],[3,4],[5,6]]) == [3,4]

In [133]:
np.mean(np.array([1,2,3,4,5,6]).reshape(3,2), axis = 0)

array([3., 4.])

In [134]:
def dot(v: Vector, w: Vector) -> float:
    '''Computes dot product'''
    assert len(v) == len(w), 'vectors must be same length'
    
    return sum(v_i * w_i for v_i, w_i in zip(v, w))
assert dot([1,2,3],[4,5,6]) == 1 * 4 + 2 * 5 + 3 * 6

In [136]:
def sum_of_squares(v: Vector) -> float:
    '''Returns sum of squares of a vector'''
    return dot(v, v)
assert sum_of_squares([1,2,3]) == 1 ** 2 + 2 ** 2 + 3 ** 2

In [140]:
def magnitude(v: Vector) -> float:
    '''Returns the magnitude (or length) of vector'''
    return math.sqrt(sum_of_squares(v))
assert magnitude([3,4]) == 5

In [141]:
def squared_distance(v: Vector, w: Vector) -> float:
    '''Returns squared distance of two vectors'''
    return sum_of_squares(subtract(v, w))

In [142]:
def distance(v: Vector, w: Vector) -> float:
    '''Returns the distance between two vectors'''
    return math.sqrt(squared_distance(v, w))

In [143]:
def distance(v: Vector, w: Vector) -> float:
    return magnitude(subtract(v, w))
assert distance([1,2],[2,3]) == math.sqrt((1-2)**2 + (2-3)**2)

### Matrices

In [144]:
Matrix = List[List[float]]

A = [[1,2,3],[4,5,6]]
B = [[1,2],[3,4],[5,6]]

In [146]:
def shape(A: Matrix) -> Tuple[int, int]:
    '''Returns number of rows and columns of matrix'''
    num_rows = len(A)
    num_cols = len(A[0]) if A else 0
    return num_rows, num_cols 
assert shape([[1,2,3],[4,5,6]]) == (2,3)

In [147]:
def get_row(A: Matrix, i: int) -> Vector:
    '''Returns the i-th row of the matrix as a vector'''
    return A[i]
def get_column(A: Matrix, i: int) -> Vector:
    '''Returns the j-th column of the matrix as a vector'''
    return [A_i[j] for A_i in A]

In [151]:
def make_matrix(num_rows: int,
                num_cols: int,
                entry_fn: Callable[[int, int], float]) -> Matrix:
    '''Returns a num_rows x num_cols matrix'''
    return [[entry_fn(i, j) 
             for j in range(num_cols)]
            for i in range(num_rows)]

In [152]:
def identity_matrix(n: int) -> Matrix:
    '''Returns the n x n identity matrix'''
    return make_matrix(n, n, lambda i, j: 1 if i == j else 0)
assert identity_matrix(3) == [[1,0,0],
                              [0,1,0],
                              [0,0,1]]

## Omitted variable bias

**OVB** =
- Effect of treatment in short - effect of treatment in long 
- beta short - beta long 
- relationship between omitted and treatment x effect of omitted in long

In [2]:
df = sm.datasets.randhie.load_pandas().data
df

,mdvis,lncoins,idp,lpi,fmde,physlm,disea,hlthg,hlthf,hlthp
0,0,4.615120,1,6.907755,0.000000,0.000000,13.73189,1,0,0
1,2,4.615120,1,6.907755,0.000000,0.000000,13.73189,1,0,0
2,0,4.615120,1,6.907755,0.000000,0.000000,13.73189,1,0,0
3,0,4.615120,1,6.907755,0.000000,0.000000,13.73189,1,0,0
4,0,4.615120,1,6.907755,0.000000,0.000000,13.73189,1,0,0
...,...,...,...,...,...,...,...,...,...,...
20185,2,0.000000,0,5.377498,0.000000,0.144292,10.57626,0,0,0
20186,0,0.000000,0,5.377498,0.000000,0.144292,10.57626,0,0,0
20187,8,3.258096,0,6.874819,8.006368,0.144292,10.57626,0,0,0
20188,8,3.258096,0,5.156178,6.542472,0.144292,10.57626,0,0,0


In [3]:
print(sm.datasets.randhie.NOTE)

::

    Number of observations - 20,190
    Number of variables - 10
    Variable name definitions::

        mdvis   - Number of outpatient visits to an MD
        lncoins - ln(coinsurance + 1), 0 <= coninsurance <= 100
        idp     - 1 if individual deductible plan, 0 otherwise
        lpi     - ln(max(1, annual participation incentive payment))
        fmde    - 0 if idp = 1; ln(max(1, MDE/(0.01 coinsurance))) otherwise
        physlm  - 1 if the person has a physical limitation
        disea   - number of chronic diseases
        hlthg   - 1 if self-rated health is good
        hlthf   - 1 if self-rated health is fair
        hlthp   - 1 if self-rated health is poor
        (Omitted category is excellent self-rated health)



In [4]:
long_lm = smf.ols(formula = 'mdvis ~ lncoins + idp', data = df).fit()
short_lm = smf.ols(formula = 'mdvis ~ lncoins', data = df).fit()
omi_lm = smf.ols(formula = 'idp ~ lncoins', data = df).fit()

In [5]:
short_lm.params['lncoins'] - long_lm.params['lncoins']

0.04094676829074495

In [6]:
omi_lm.params['lncoins'] * long_lm.params['idp']

0.04094676829074513

In [7]:
long_lm = smf.ols(formula = 'mdvis ~ lncoins + idp + lpi + fmde', data = df).fit()
short_lm = smf.ols(formula = 'mdvis ~ lncoins + lpi + fmde', data = df).fit()
omi_lm = smf.ols(formula = 'idp ~ lncoins + lpi + fmde', data = df).fit()

In [8]:
short_lm.params['lncoins'] - long_lm.params['lncoins']

0.06802680182337317

In [9]:
omi_lm.params['lncoins'] * long_lm.params['idp']

0.06802680182337166

## Linearity

## Bivariate regression and covariance

In [10]:
bi_lm = smf.ols(formula = 'mdvis ~ lncoins', data = df).fit()
bi_lm.params

Intercept    3.141148
lncoins     -0.158236
dtype: float64

In [11]:
np.mean(df.mdvis) - bi_lm.params['lncoins'] * np.mean(df.lncoins)

3.1411484061092625

In [12]:
np.cov(np.array([df.mdvis, df.lncoins]))[0][1] / np.var(df.lncoins)

-0.1582441097715173

## Residuals

In [13]:
lm = smf.ols(formula = 'mdvis ~ lncoins + idp + lpi + fmde', data = df).fit()
lm.resid

0       -2.644642
1       -0.644642
2       -2.644642
3       -2.644642
4       -2.644642
           ...   
20185   -1.846313
20186   -3.846313
20187    5.408944
20188    5.414937
20189    3.435715
Length: 20190, dtype: float64

### Mean zero

In [14]:
lm.resid.mean().round(decimals = 10)

0.0

In [15]:
df_new = df.assign(resid = lm.resid, pre = lm.predict()) 
df_new

,mdvis,lncoins,idp,lpi,fmde,physlm,disea,hlthg,hlthf,hlthp,resid,pre
0,0,4.615120,1,6.907755,0.000000,0.000000,13.73189,1,0,0,-2.644642,2.644642
1,2,4.615120,1,6.907755,0.000000,0.000000,13.73189,1,0,0,-0.644642,2.644642
2,0,4.615120,1,6.907755,0.000000,0.000000,13.73189,1,0,0,-2.644642,2.644642
3,0,4.615120,1,6.907755,0.000000,0.000000,13.73189,1,0,0,-2.644642,2.644642
4,0,4.615120,1,6.907755,0.000000,0.000000,13.73189,1,0,0,-2.644642,2.644642
...,...,...,...,...,...,...,...,...,...,...,...,...
20185,2,0.000000,0,5.377498,0.000000,0.144292,10.57626,0,0,0,-1.846313,3.846313
20186,0,0.000000,0,5.377498,0.000000,0.144292,10.57626,0,0,0,-3.846313,3.846313
20187,8,3.258096,0,6.874819,8.006368,0.144292,10.57626,0,0,0,5.408944,2.591056
20188,8,3.258096,0,5.156178,6.542472,0.144292,10.57626,0,0,0,5.414937,2.585063


### Uncorrelated with regressors and predictions

In [16]:
resid_lm = smf.ols(formula = 'resid ~ lncoins + idp + lpi + fmde', data = df_new).fit()
resid_lm.params.round(decimals = 10)

Intercept   -0.0
lncoins      0.0
idp          0.0
lpi         -0.0
fmde         0.0
dtype: float64

In [17]:
pre_lm = smf.ols(formula = 'resid ~ pre', data = df_new).fit()
pre_lm.params.round(decimals = 10)

Intercept    0.0
pre         -0.0
dtype: float64

## Dummies

### Dummies in bivariate regression

In [18]:
dum_lm = smf.ols(formula = 'mdvis ~ idp', data = df).fit()
dum_lm.params

Intercept    2.996453
idp         -0.523220
dtype: float64

In [19]:
df[df['idp'] == 0].mdvis.mean()

2.9964527140084334

In [20]:
df[df['idp'] == 1].mdvis.mean() - df[df['idp'] == 0].mdvis.mean()

-0.5232197172471453

## Regression anatomy

### Parameters

In [21]:
ana_lm = smf.ols(formula = 'mdvis ~ lncoins + idp + lpi', data = df).fit()
ana_res_lm = smf.ols(formula = 'lncoins ~ idp + lpi', data = df).fit()

In [22]:
ana_lm.params

Intercept    3.218183
lncoins     -0.236384
idp         -0.832246
lpi          0.059044
dtype: float64

In [23]:
np.cov(df.mdvis, ana_res_lm.resid)[0][1] / np.var(ana_res_lm.resid)

-0.2363961829157149

## Logs

In [35]:
log_lm = smf.ols(formula = 'np.log(mdvis) ~ lncoins', data = df[df.mdvis != 0]).fit()
log_lm.params['lncoins'].round(3) * 100

-2.7

In [40]:
(log_lm.predict(exog = {'lncoins':2.5}) - log_lm.predict(exog = {'lncoins':1.5})) / log_lm.predict(exog = {'lncoins':1.5}).round(3) * 100

0   -2.600802
dtype: float64

## Standard error and confidence interval

### SE formula for bivariate regression

In [73]:
seb_lm = smf.ols(formula = 'mdvis ~ lncoins', data = df).fit()

In [74]:
seb_lm.params

Intercept    3.141148
lncoins     -0.158236
dtype: float64

In [75]:
seb_lm.bse

Intercept    0.042431
lncoins      0.015946
dtype: float64

In [76]:
seb_lm.conf_int()

,0,1
Intercept,3.057981,3.224316
lncoins,-0.189491,-0.126981


In [77]:
np.std(seb_lm.resid) / np.sqrt(len(df)) / np.std(df.lncoins)

0.015945074249450204

### SE formula for multivariate regression

In [78]:
sem_lm = smf.ols(formula = 'mdvis ~ lncoins + idp + lpi', data = df).fit()

In [79]:
sem_lm.params

Intercept    3.218183
lncoins     -0.236384
idp         -0.832246
lpi          0.059044
dtype: float64

In [80]:
sem_lm.bse

Intercept    0.065508
lncoins      0.018399
idp          0.076638
lpi          0.013200
dtype: float64

In [81]:
sem_lm.conf_int()

,0,1
Intercept,3.089783,3.346584
lncoins,-0.272447,-0.200321
idp,-0.982462,-0.682030
lpi,0.033171,0.084917


In [82]:
sem_res_lm = smf.ols(formula = 'lncoins ~ idp + lpi', data = df).fit()

In [84]:
np.std(sem_lm.resid) / np.sqrt(len(df)) / np.std(sem_res_lm.resid)

0.018396901253716645

## RSS - Residual sum of squares

In [97]:
lm = smf.ols(formula = 'mdvis ~ lncoins + idp + lpi', data = df).fit()

In [90]:
lm.ssr

405198.90608648356

In [96]:
((lm.predict() - df.mdvis) ** 2).sum()

405198.9060864836

## Plots

In [104]:
sm.graphics

<module 'statsmodels.graphics.api' from '/Users/boyuan/anaconda3/envs/sds/lib/python3.8/site-packages/statsmodels/graphics/api.py'>